In [ ]:
import os
import sys
import json
import glob
import shutil
import deepdiff
import datetime
import rasterio
import numpy as np

sys.path.insert(0, '../')
from managers import managers

%load_ext autoreload
%autoreload 2

### Test DEM

In [ ]:
# DEM test
proj = managers.DEMProject(
    project_root='../projects/test-dem', 
    dataset_paths='../test/datasets/dem/n38w120/n38w120_100m.tif',
    reset=True)

proj.hill_shade(proj.get_operation('last', 'merge'))
proj.slope_shade(proj.get_operation('last', 'merge'))
proj.texture_shade(proj.get_operation('last', 'merge'))
proj.save_props()

In [ ]:
proj._serialize()

In [ ]:
# DEM test
proj = managers.DEMProject(
    project_root='../projects/test-dem', 
    dataset_paths='../test/datasets/dem/n38w120/n38w120_100m.tif',
    reset=False)

In [ ]:
proj._serialize()

### Test Landsat

In [ ]:
# test 'raw' landsat scenes
test_scenes = [
    '../test/datasets/landsat/LC08_L1TP_041034_20180916_20180928_01_T1',
    '../test/datasets/landsat/LC08_L1TP_042034_20180907_20180912_01_T1',]

In [ ]:
# real raw landsat scenes
full_scenes = [
    '/home/keith/landsat-data//LC08_L1TP_041034_20180916_20180928_01_T1',
    '/home/keith/landsat-data//LC08_L1TP_042034_20180907_20180912_01_T1',]

In [ ]:
# bounds for central Sierra 
latlon_bounds = [-119.8, 37.2, -117.8, 38.2]

In [ ]:
# check that we can reproduce these transformed bounds
landsat_crs_bounds = [251494.21, 4120732.28, 429952.78, 4228308.24]

tif = '../test/datasets/landsat/LC08_L1TP_041034_20180916_20180928_01_T1/LC08_L1TP_041034_20180916_20180928_01_T1_B1.TIF'
bounds = managers.utils.transform(latlon_bounds, tif)

assert(bounds==landsat_crs_bounds)

In [ ]:
# test landsatProject *with* resetting
proj = managers.LandsatProject(
    project_root='../projects/test-landsat',
    dataset_paths=test_scenes,
    res=400,
    bounds=bounds,
    reset=True)

In [ ]:
proj.stack(proj.operations[0].destination, bands=[7, 5, 2])

In [ ]:
proj.autogain(proj.operations[-1].destination, percentile=99)

In [ ]:
# load without resetting
proj = managers.LandsatProject(
    project_root='../projects/test-landsat',
    reset=False)

In [ ]:
proj.operations

In [ ]:
proj._serialize()

In [ ]:
proj.save_props()

In [ ]:
# load real raw datasets
proj = managers.LandsatProject(
    project_root='/home/keith/landsat-data/projects/mono-lake/',
    dataset_paths=full_scenes,
    bounds=bounds,
    reset=False)

In [ ]:
# test operation class
s = managers.datasets.new_dataset('tif', 'test/path/one')
d = managers.datasets.new_dataset('tif', 'test/path/two')
op = managers.Operation(s, d, 'method_name', {'key': None}, 'commit_hash')

In [ ]:
managers.datasets.new_dataset('ned13', '../test/datasets/dem/n38w120/n38w120_100m.tif')

In [ ]:
managers.datasets.new_dataset('tif', '/media/keith/USGS_Backup/USGS/NED13/n39w123/', exists=True)